# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd

from src import CTX, SEED, FOLDERS
from src.features import COLUMNS, build_features

In [3]:
from src.data import io
observations = io.load_data(
    FOLDERS.PROCESSED,
    CTX + 'observations',
    pd.DataFrame())
observations

Loading ../data/processed/CC-Labs-hv_observations.h5


,Observation
0,"Tens of categories with very few items, a hand..."
1,First word of category name a good candidate f...
2,(Categories already sorted on category name)
3,First word of shop name a good candidate for s...
4,(Shops already sorted on shop name)
5,Submissions are evaluated by root mean squared...
6,"All shop, item, and category references are va..."
7,Training data does not have explicit NULLs
8,For each ID in the test set (shop_id/item_id c...
9,One in seven test data keys do not have entrie...


# Read raw data

In [4]:
from src.data import io
sales, test, items, categories, shops = io.read_raw(FOLDERS.RAW)

In [5]:
categories = build_features.add_super_category(categories)
shops = build_features.add_super_shop(shops)
train = build_features.enrich_sales(
    build_features.rollup_and_clip_sales(sales),
    shops, items, categories)
train.head()

,date_block_num,shop_id,item_id,item_cnt_month,shop_name,super_shop_name,super_shop_id,item_name,item_category_id,item_category_name,super_category_name,super_category_id
0,0,0,32,6.0,"!Якутск Орджоникидзе, 56 фран",!Якутск,0,1+1,40,Кино - DVD,Кино,7
1,0,0,33,3.0,"!Якутск Орджоникидзе, 56 фран",!Якутск,0,1+1 (BD),37,Кино - Blu-Ray,Кино,7
2,0,0,35,1.0,"!Якутск Орджоникидзе, 56 фран",!Якутск,0,10 ЛЕТ СПУСТЯ,40,Кино - DVD,Кино,7
3,0,0,43,1.0,"!Якутск Орджоникидзе, 56 фран",!Якутск,0,100 МИЛЛИОНОВ ЕВРО,40,Кино - DVD,Кино,7
4,0,0,51,2.0,"!Якутск Орджоникидзе, 56 фран",!Якутск,0,100 лучших произведений классики (mp3-CD) (Dig...,57,Музыка - MP3,Музыка,9


# Create grid

In [10]:
max_train_date_block_num = train.date_block_num.max()
max_train_date_block_num

33

In [32]:
from tqdm import tqdm_notebook
all_data = build_features.create_grid(
    pd.concat([
        train.rename(columns={'item_cnt_month' : 'target'}).drop(columns=['shop_name', 'super_shop_name', 'item_name', 'item_category_name', 'super_category_name']),
        test.assign(date_block_num=max_train_date_block_num+1)],
        ignore_index=True, sort=False),
    COLUMNS.KEYS_AND_TIME,
    tqdm_notebook)
print(len(all_data))
all_data.head()

/home/mortennp/miniconda3/envs/comp-ds-3_6/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


11128050


,date_block_num,shop_id,item_id,target,super_shop_id,item_category_id,super_category_id,ID,target_shop,target_super_shop,target_item,target_category,target_super_category
0,0,0,32,6.0,0.0,40.0,7.0,0.0,5411.0,8343.0,242.0,31656.0,41060.0
1,0,0,33,3.0,0.0,37.0,7.0,0.0,5411.0,8343.0,61.0,5982.0,41060.0
2,0,0,35,1.0,0.0,40.0,7.0,0.0,5411.0,8343.0,78.0,31656.0,41060.0
3,0,0,43,1.0,0.0,40.0,7.0,0.0,5411.0,8343.0,2.0,31656.0,41060.0
4,0,0,51,2.0,0.0,57.0,9.0,0.0,5411.0,8343.0,13.0,1539.0,15405.0


In [33]:
from src.data import io
io.save_data(
    FOLDERS.PROCESSED,
    CTX + 'grid',
    all_data)

Saving ../data/processed/CC-Labs-hv_grid.h5


# Create lags

In [34]:
lagged_data, to_drop_cols = build_features.create_lags(
    all_data,
    COLUMNS.KEYS_AND_TIME,
    tqdm_notebook)
lagged_data.head()
to_drop_cols

NameError: name 'item_category_mapping' is not defined

In [ ]:
del all_data
gc.collect()